# Preprocessing data prior to machine learning

For machine learning, we assume that data is in shape `(n_cases, n_channels,
n_timepoints)` for equal length series or a python list with `len` of `[n_cases]` if
the series are unequal length. However, in reality, there are often many steps to get
 your data into this format. We introduce some common uses cases that may be handled
 with preprocessing, and give some suggestions about how to handle them.


## Missing values

Missing values are indicated by `NaN` in numpy array. You can test whether any `aeon`
 data structure contains missing values using the utility function



## Unequal length series



## Unequally spaced samples



## Streaming series: windowing and segmenting




# Channel selection for multivariate series

Elbow points

# Condensing the training set

Averaging etc.